如今人类和机器都能很好地区分猫和狗：这是因为图像中本就拥有丰富的结构，而这些结构可以被人类和机器学习模型使用。  
卷积神经网络（convolutional neural networks，CNN）是机器学习利用自然图像中一些已知结构的创造性方法

图像信息具有以下性质：  

平移不变性（translation invariance）：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应，即为“平移不变性”。  

局部性（locality）：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系，这就是“局部性”原则。最终，可以聚合这些局部特征，以在整个图像级别进行预测。  


互相关运算  
接下来，我们在corr2d函数中实现如上过程，该函数接受输入张量X和卷积核张量K，并返回输出张量Y

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  #@save
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y


In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)



tensor([[19., 25.],
        [37., 43.]])

卷积层  
卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。  
所以，卷积层中的两个被训练的参数是卷积核权重和标量偏置。  
就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。

基于上面定义的corr2d函数实现二维卷积层。 
在__init__构造函数中，将weight和bias声明为两个模型参数。前向传播函数调用corr2d函数并添加偏置。

In [3]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))
    def forward(self,X):
        return corr2d(X,self.weight)+self.bias

图像中目标的边缘检测  
通过找到像素变化的位置，来检测图像中不同颜色的边缘  
构造一个像素6*8的黑白图像。  
中间四列为黑色（0），其余像素为白色（1）

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
K = torch.tensor([[1.0, -1.0]])


In [6]:
Y = corr2d(X, K)
Y


tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

现在我们将输入的二维图像转置，再进行如上的互相关运算。  
其输出如下，之前检测到的垂直边缘消失了。  
不出所料，这个卷积核K只可以检测垂直边缘，无法检测水平边缘。

In [7]:
corr2d(X.t(), K)


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习卷积核  
学习由X生成Y的卷积核  
先构造一个卷积层，并将其卷积核初始化为随机张量。  
接下来，在每次迭代中，我们比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。  
为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

In [12]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

X=X.reshape(1,1,6,8)
Y=Y.reshape(1,1,6,7)
lr=3e-2

for i in range(100):
    Y_hat=conv2d(X)
    l=(Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()

    conv2d.weight.data[:]-=lr*conv2d.weight.grad
    if (i+1)%20==0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')


epoch 20, loss 0.001
epoch 40, loss 0.000
epoch 60, loss 0.000
epoch 80, loss 0.000
epoch 100, loss 0.000


In [13]:
conv2d.weight.data.reshape((1, 2))


tensor([[ 1.0000, -1.0000]])

填充（padding）和步幅（stride）  
输入矩阵形状$n_h \times n_w$  
卷积核大小$k_h \times k_w$  
输出形状$(n_h-(k_h-1))\times(n_w-(k_w-1))$  
如果在高和宽方向分别填充$2p_h \quad 2p_w$,两侧均匀填充$p_h \quad p_w$则输出形状为：  
$(n_h-(k_h-1)+2p_h)\times(n_w-(k_w-1)+2p_w)$  
当设置$(k_h-1)=2p_h \quad (k_w-1)=2p_w$时，输出与输入形状相同  
等价于在两侧都填充
$$p_h=\frac{k_h-1}{2} \text{行} \\ p_w=\frac{k_w-1}{2} \text{行}$$
当$k_h$与$k_w$均为奇数时，方便使得输入输出形状相同且两侧均匀填充  
卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7  

<pre>对于任何二维张量X，当满足： 
1. 卷积核的大小是奇数； 
2. 所有边的填充行数和列数相同； 
3. 输出与输入具有相同高度和宽度；
则：
输出Y[i, j]是通过以输入X[i, j]为中心，与卷积核进行互相关计算得到的

In [ ]:
# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)#（1,1,（X.shape））(批量，通道，高，宽)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape


torch.Size([8, 8])

当卷积核的高度和宽度不同时，我们可以填充不同的高度和宽度，使输出和输入具有相同的高度和宽度。  
在如下示例中，我们使用高度为5，宽度为3的卷积核，高度和宽度两边的填充分别为2和1。

In [16]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape


torch.Size([8, 8])

步幅  
将每次滑动元素的数量称为步幅（stride）  
当垂直与水平步幅分别为$s_h \quad s_w$时，输出形状为  
$[(n_h-(k_h-1)+2p_h+s_h-1)/s_h]\times[(n_w-(k_w-1)+2p_w+s_w-1)/s_w]$  
当$p_h=\frac{k_h-1}{2} \text{行} \quad p_w=\frac{k_w-1}{2} \text{行}$时形状为  
$[(n_h+s_h-1)/s_h]\times[(n_w+s_w-1)/s_w]$  


将高度和宽度的步幅设置为2，从而将输入的高度和宽度减半。

In [17]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape


torch.Size([4, 4])

稍微复杂的例子

In [18]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape


torch.Size([2, 2])

在实践中，我们很少使用不一致的步幅或填充，也就是说，我们通常有  
$p_h=p_w=p$和$s_h=s_w=s$